In [1]:
import numpy as np
import numpy.testing as npt

Initially, I'll write a function that calculates probabilities of classes in terminal nodes.

In [11]:
class ProbabilityMatcher():
    def __init__(self,n_classes):
        self.num_of_classes=n_classes
    
    def get_proba(self, y):

        return np.bincount(y,minlength=self.num_of_classes)/y.size

In [3]:
a=np.array([1,1,2,4,2,2,0,1,0,4])

In [4]:
p=ProbabilityMatcher(n_classes=6)
assert p.get_proba(a).tolist()==[0.2, 0.3, 0.3, 0. , 0.2, 0. ]

In [5]:
class BaseInformationCriterion:
    
    def find_best_split(self,X_feature,y):
        thresholds=sorted(list(set(X_feature)))[1:-1]
        max_q={'threshold':None,'Q_max':-np.inf}
        H_main=get_H(y)
        for treshold in thresholds:
            mask = X_feature>=threshold
            y_right=y[mask]
            y_left=y[~mask]
            Q=get_Q(H_main,y_left,y_right)
            if Q>max_q['max']:
                max_q['threshold']=threshold
                max_q['Q_max']= Q
        return max_q.values()
            
    def get_Q(self,H_main,y_left,y_right):
        len_left=len(y_left)
        len_right=len(y_right)
        total=len_left+len_right
        if len_left==0:
            return H_main-len_right/total*self.get_H(y_right)
        if len_right==0:
            return H_main-len_left/total*self.get_H(y_left)
        return H_main-len_left/total*self.get_H(y_left)-len_right/total*self.get_H(y_right)
    
    def get_H(self,y):
        raise NotImplementedError
        
        
        
        
class GiniCriterion(BaseInformationCriterion):
    def __init__(self,n_classes=None):
        self.probs_matcher=ProbabilityMatcher(n_classes)
    def get_H(self,y):
        probs=self.probs_matcher.get_proba(y)
        antiprobs=1-probs
        return (probs*antiprobs).sum()

In [9]:
g=GiniCriterion()
h_main=g.get_H(a)
npt.assert_almost_equal(h_main,0.74,4)

In [15]:

npt.assert_almost_equal(
    g.get_Q(h_main,np.array([1,1,1,0]),np.array([2,2,0,2,4,4])),0.2233333333,decimal=4)